# install

In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.3 MB/s eta 0:00:00


# buat eksperimen
yg dilakukan pd tahap ini adalah
* melatih model
* hyperparameter tuning
* tracking training dan hyperparameternya menggunakan mlflow sehingga mlflow akan membuat artifact nya (isinya training, data dan konfigurasi hyperparameter)

In [ ]:
# membuat eksperimen yg dpt ditracking oleh mlflow
import mlflow

# artifact akan disimpan di directory experiment_artifact
# artifact isinya bisa berupa training, hyperparameter, evaluasi, confusion matrix, file hasil training (untuk fine tuning), dll
!mkdir experiment_artifact

# buat eksperimen menggunakan mlflow
mlflow.create_experiment(
    name = "experimen_1",
    artifact_location='experiment_artifact/',
    tags = {"env":"dev", "version":"1.0.0"},
)

'486176321809614432'

ketika dirunning akan generate direktori mlruns/ isinya ada 0 (id dari eksperimen, dibuat scra otomatis oleh mlflow dan start defaultnya 0) dan 486176321809614432 (eksperimen yg dilakukan)

isi dari mlruns/0/meta.yaml adlh

    artifact_location: file:///content/mlruns/0
    creation_time: 1703677323276
    experiment_id: '0'
    last_update_time: 1703677323276
    lifecycle_stage: active
    name: Default

name nya Default karena id dibuat oleh mlflow

isi dari mlruns/48617.../meta.yaml (artifact dari ekperimen yg kita lakukan)

    artifact_location: /content/experiment_artifact
    creation_time: 1703677323286
    experiment_id: '486176321809614432'
    last_update_time: 1703677323286
    lifecycle_stage: active
    name: experimen_1

selain itu ada juga `/content/mlruns/486176321809614432/tags/env` dan `/content/mlruns/486176321809614432/tags/version` yg merupakan isi dari tags ketika kita pertama membuat experiment

untuk melihat eksperimen dlm ui maka gunakan perintah berikut di terminal

    mlflow ui




In [ ]:
!mlflow ui

[2023-12-27 11:49:13 +0000] [4461] [INFO] Starting gunicorn 21.2.0
[2023-12-27 11:49:13 +0000] [4461] [INFO] Listening at: http://127.0.0.1:5000 (4461)
[2023-12-27 11:49:13 +0000] [4461] [INFO] Using worker: sync
[2023-12-27 11:49:13 +0000] [4462] [INFO] Booting worker with pid: 4462
[2023-12-27 11:49:14 +0000] [4463] [INFO] Booting worker with pid: 4463
[2023-12-27 11:49:14 +0000] [4464] [INFO] Booting worker with pid: 4464
[2023-12-27 11:49:14 +0000] [4465] [INFO] Booting worker with pid: 4465

Aborted!
[2023-12-27 11:49:54 +0000] [4461] [INFO] Handling signal: int
[2023-12-27 11:49:54 +0000] [4462] [INFO] Worker exiting (pid: 4462)
[2023-12-27 11:49:54 +0000] [4465] [INFO] Worker exiting (pid: 4465)
[2023-12-27 11:49:54 +0000] [4464] [INFO] Worker exiting (pid: 4464)
[2023-12-27 11:49:54 +0000] [4463] [INFO] Worker exiting (pid: 4463)
[2023-12-27 11:49:55 +0000] [4461] [INFO] Shutting down: Master


agar lbh baik struktur kodenya maka kita buat fungsi eksperimen

In [ ]:
import mlflow
from typing import Any

def create_experiment_mlflow(experiment_name: str, artifact_save_location: str, tags: dict[str, Any]) -> str:
  # jika experiment blm pernah dibuat
  try:
    experiment_id = mlflow.create_experiment(
        name = experiment_name, artifact_location = artifact_save_location, tags = tags
    )
  # jika experiment sdh pernah dibuat
  except:
    print(f"Experiment {experiment_name} already exists.")
    # dptkan id dari experiment nya
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

  return experiment_id

In [ ]:
experiment_id = create_experiment_mlflow(
    experiment_name = "experimen_1",
    artifact_save_location ='experiment_artifact/',
    tags = {"env":"dev", "version":"1.0.0"},
)
print(experiment_id)

Experiment experimen_1 already exists.
486176321809614432


In [ ]:
experiment_id = create_experiment_mlflow(
    experiment_name = "experimen_2",
    artifact_save_location ='experiment_artifact/',
    tags = {"env":"dev", "version":"1.0.0"},
)
print(experiment_id)

531631867608238165


# Retrieving Experiments
mendapatkan informasi tentang eksperimen yg tlh dibuat.

In [ ]:
import mlflow
# untuk mendptkan informasi tentang eksperimen, mlflow perlu tahu id dari eksperimennya atau nama dari eksperimennya
# sehingga kedua data tsb jd parameter pd fungsi yg kita buat
def get_mlflow_experiment(experiment_id: str = None, experiment_name: str = None) -> mlflow.entities.Experiment:
    # get informasi id eksperimen
    if experiment_id is not None:
        experiment = mlflow.get_experiment(experiment_id)
    # get informasi nama eksperimen
    elif experiment_name is not None:
        experiment = mlflow.get_experiment_by_name(experiment_name)
    # eksperiment tdk ada
    else:
        raise ValueError("Either experiment_id or experiment_name must be provided.")
    return experiment

In [ ]:
# cari menggunakan id
experiment = get_mlflow_experiment(experiment_id="486176321809614432")

print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
print("Creation timestamp: {}".format(experiment.creation_time))

Name: experimen_1
Experiment_id: 486176321809614432
Artifact Location: /content/experiment_artifact
Tags: {'version': '1.0.0', 'env': 'dev'}
Lifecycle_stage: active
Creation timestamp: 1703677323286


In [ ]:
# cari menggunakan nama
experiment = get_mlflow_experiment(experiment_name="experimen_1")

print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
print("Creation timestamp: {}".format(experiment.creation_time))

Name: experimen_1
Experiment_id: 486176321809614432
Artifact Location: /content/experiment_artifact
Tags: {'version': '1.0.0', 'env': 'dev'}
Lifecycle_stage: active
Creation timestamp: 1703677323286


# Deleting experiment
menghapus eksperimen yg tlh dibuat.

In [ ]:
import mlflow

# hapus eksperimen dg id yg kita berikan
mlflow.delete_experiment(experiment_id='531631867608238165')

maka direktori mlruns/531631867608238165 akan dihapus

# MLflow runs

## inisialisasi/memulai (start) mlflor run
untuk mendptkan log maka kita perlu menjlnkan mlflow runs

stlh itu baru kita dpt menyimpan log (misalnya parameter) kedlm eksperimen

selain parameter kita bisa memasukkan description, metrics, tags, artifact kedlm eksperimen

In [ ]:
import mlflow
# inisialisasi mlflow runs
mlflow.start_run()

# informasi log parameter misalnya learning rate dimasukkan kedlm eksperimen
mlflow.log_param("learning_rate", 0.01)

# akhiri mlflow runs nya
mlflow.end_run()

## inisialisasi/memulai (start) mlflor run serta menggunakan with
yg didlm indentasi with akan masuk kedlm log dari mlflow jd kode lbh simple

In [ ]:
import mlflow
with mlflow.start_run(run_name="mlflow_run") as run:
  # log parameter
  mlflow.log_param('learning_rate',0.01)
  print('Run id')
  print(run.info.run_id)
  print('='*10)
  print(run.info)

Run id
c64fddea6ef040fba0502cf74ab0ee02
<RunInfo: artifact_uri='file:///content/mlruns/0/c64fddea6ef040fba0502cf74ab0ee02/artifacts', end_time=None, experiment_id='0', lifecycle_stage='active', run_id='c64fddea6ef040fba0502cf74ab0ee02', run_name='mlflow_run', run_uuid='c64fddea6ef040fba0502cf74ab0ee02', start_time=1703680170426, status='RUNNING', user_id='root'>


## tracking eksperimen


In [ ]:
if __name__=="__main__":

    # buat eksperimen
    experiment_id = create_experiment_mlflow(
        experiment_name = "testing_mlflow1",
        artifact_save_location ="testing_mlflow1_artifacts",
        tags={"env": "dev", "version": "1.0.0"},
    )

    # cari informasi dari eksperimen yg kita berikan menggunakan name
    mlflow.set_experiment(experiment_name='testing_mlflow1')

    # jlnkan mlflow run
    with mlflow.start_run(run_name="testing") as run:

        # masukkan data learning rate kedlm log parameter
        mlflow.log_param("learning_rate",0.01)
        # run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

run_id: 15fecba058bc4a1f8687af719a8bc608
experiment_id: 501789625614987955
status: RUNNING
start_time: 1703680756382
end_time: None
lifecycle_stage: active


sekarang kita lihat menggunakan id dari eksperimen

In [ ]:
if __name__=="__main__":

    # buat eksperimen
    experiment_id = create_experiment_mlflow(
        experiment_name = "testing_mlflow1",
        artifact_save_location ="testing_mlflow1_artifacts",
        tags={"env": "dev", "version": "1.0.0"},
    )

    # jlnkan mlflow run
    with mlflow.start_run(run_name="testing", experiment_id = experiment.experiment_id) as run:

        # masukkan data learning rate kedlm log parameter
        mlflow.log_param("learning_rate",0.01)
        # run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Experiment testing_mlflow1 already exists.
run_id: 06ad669fb81140e3a97e401bde307e83
experiment_id: 501789625614987955
status: RUNNING
start_time: 1703680884753
end_time: None
lifecycle_stage: active


cari berdsrkan id eksperimen yg kita buat

In [ ]:
if __name__=="__main__":

    # buat eksperimen
    experiment_id = create_experiment_mlflow(
        experiment_name = "testing_mlflow1",
        artifact_save_location ="testing_mlflow1_artifacts",
        tags={"env": "dev", "version": "1.0.0"},
    )

    # dptkan informasi id dari eksperimen
    experiment = get_mlflow_experiment(experiment_id=experiment_id)
    print("Name: {}".format(experiment.name))

    # jlnkan mlflow run
    with mlflow.start_run(run_name="testing", experiment_id = experiment.experiment_id) as run:

        # masukkan data learning rate kedlm log parameter
        mlflow.log_param("learning_rate",0.01)
        # run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: a7516f6584164744845f3f8e4037ee2b
experiment_id: 172815588612644609
status: RUNNING
start_time: 1703691622188
end_time: None
lifecycle_stage: active


## logging parameters
memasukkan hyperparameter selama training kedlm mlflow

In [ ]:
if __name__ == "__main__":

    # dptkan informasi dari eksperimen yg tlh kita buat
    experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")
    print("Name: {}".format(experiment.name))

    # jlnkan mlflow run pd id dari eksperimen yg kita inginkan
    with mlflow.start_run(run_name="logging_params", experiment_id = experiment.experiment_id) as run:

        # hanya 1 log parameter yg dimasukkan kedlm mlflow yaitu hyperparameter learning rate
        mlflow.log_param("learning_rate", 0.01)

        # setting semua hyperparameter
        parameters = {
            "learning_rate": 0.01,
            "epochs": 10,
            "batch_size": 100,
            "loss_function": "mse",
            "optimizer": "adam"
        }

        # masukkan semua hyperparameter kedlm mlflow melalui log param dimana parameternya adlh semua hyperparameter yg tlh kita setting diatas (bntknya dictionary)
        mlflow.log_params(parameters)

        # print run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: 0322780551bc463b9b203d9b9dcc3134
experiment_id: 501789625614987955
status: RUNNING
start_time: 1703681276533
end_time: None
lifecycle_stage: active


## Logging Metrics
metrics disini adlh pengukuran kinerja dari model / evaluasi model sprti matrix : mse, akurasi, recall, dll

In [ ]:
if __name__=="__main__":

    experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")
    print("Name: {}".format(experiment.name))

    with mlflow.start_run(run_name="logging_metrics", experiment_id = experiment.experiment_id) as run:

        # masukkan kedlm mlflow 1 log metrics yaitu random_metrics
        mlflow.log_metric("random_metric", 0.01)

        # setting metrics (dlm bntk dictionary)
        metrics = {
            "mse": 0.01,
            "mae": 0.01,
            "rmse": 0.01,
            "r2": 0.01
        }

        # masukkan kedlm mlflow semua metrics tsb yg sblm tlh kita setting
        mlflow.log_metrics(metrics)

        # print run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: 842fc31bb0b44024a5f043329b61261a
experiment_id: 501789625614987955
status: RUNNING
start_time: 1703681579540
end_time: None
lifecycle_stage: active


## Logging artifacts

### menyimpan file ke artifact dlm mlflow
artifact bisa berupa file hsl training, file yg dihslkan oleh model (hsl prediksi dlm bntk file)

In [ ]:
if __name__=="__main__":
    experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")

    print("Name: {}".format(experiment.name))

    with mlflow.start_run(run_name="logging_artifacts", experiment_id=experiment.experiment_id) as run:

        # buat file bernama hello_world.txt yg isinya adlah string hello world
        with open("hello_world.txt", "w") as f:
            f.write("Hello World!")

        # masukkan file tsb kedlm artifact mlflow dimana local_path adlh path dari file-nya,
        # artifact_path adlh nama path/direktori tempat artifact disimpan didlm mlflow
        mlflow.log_artifact(local_path="hello_world.txt", artifact_path="text_files")

        # print run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: 3e7b63a06f8f427ca0b18c8056b7e293
experiment_id: 172815588612644609
status: RUNNING
start_time: 1703691639921
end_time: None
lifecycle_stage: active


### file yg disimpannya banyak
jika sblmnya file yg tersimpan kedlm log artifact mlflow hanya satu, maka sekarang file nya lbh dr 1

In [ ]:
# file nya
!wget https://raw.githubusercontent.com/manuelgilm/mlflow_for_ml_dev/master/run_artifacts/confusion_matrix.jpeg
with open("hello_world.txt", "w") as f:
  f.write("Hello World!")
!mkdir run_artifacts
!mv /content/confusion_matrix.jpeg run_artifacts/
!mv hello_world.txt run_artifacts/

--2023-12-27 15:50:51--  https://raw.githubusercontent.com/manuelgilm/mlflow_for_ml_dev/master/run_artifacts/confusion_matrix.jpeg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17315 (17K) [image/jpeg]
Saving to: ‘confusion_matrix.jpeg’

confusion_matrix.jp 100%[===================>]  16.91K  --.-KB/s    in 0s      

2023-12-27 15:50:51 (75.2 MB/s) - ‘confusion_matrix.jpeg’ saved [17315/17315]



In [ ]:
if __name__=="__main__":

    experiment = get_mlflow_experiment(experiment_name="testing_mlflow1")

    print("Name: {}".format(experiment.name))

    with mlflow.start_run(run_name="logging_artifacts", experiment_id=experiment.experiment_id) as run:

        # masukkan semua file yg ada di folder run_artifacts kedlm log artifact mlflow.
        # semua artifacts akan ada di mlflow directory run_artifacts
        mlflow.log_artifacts(local_dir="./run_artifacts",artifact_path="run_artifacts")

        # print run info
        print("run_id: {}".format(run.info.run_id))
        print("experiment_id: {}".format(run.info.experiment_id))
        print("status: {}".format(run.info.status))
        print("start_time: {}".format(run.info.start_time))
        print("end_time: {}".format(run.info.end_time))
        print("lifecycle_stage: {}".format(run.info.lifecycle_stage))

Name: testing_mlflow1
run_id: 4d838755e8004aabb6979dfd2937d2ab
experiment_id: 172815588612644609
status: RUNNING
start_time: 1703692386308
end_time: None
lifecycle_stage: active


## Logging images